In [ ]:
import boto3
import glob
import os
import json
from concurrent import futures

In [ ]:
aws_credentials = json.load(open('/root/thomas/aws_credentials.json'))

In [ ]:
s3 = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")

In [ ]:
files = glob.glob('/root/data/fish_detections/aquabyte-crops/environment=production/site-id=23/pen-id=4/*/*/*')

In [ ]:
todownload = []
for f in files:
    prefix = '/'.join(f.split('/')[5:])
    for s3file in ['crops.json', 'left_frame.resize_512_512.jpg', 'right_frame.resize_512_512.jpg']:
        todownload.append(os.path.join(prefix, s3file))

In [ ]:
# s3_object_keys = []
# for url in renamed_urls:
#     key = '/'.join(url.split('/')[4:])
#     s3_object_keys.append(key)

In [ ]:
path = '/root/data/fish_detections/aquabyte-frames-resized-inbound/'
bucket_name = 'aquabyte-frames-resized-inbound'
s3_object_keys = todownload

In [ ]:
max_workers = 30

# abs_path = os.path.abspath(relative_path)

def fetch(key):
    file = '{}/{}'.format(path, key)
    os.makedirs(os.path.dirname(file), exist_ok=True)  
    with open(file, 'wb') as data:
        s3.download_fileobj(bucket_name, key, data)
    return file


def fetch_all(keys):

    with futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_key = {executor.submit(fetch, key): key for key in keys}

        print("All URLs submitted.")

        for future in futures.as_completed(future_to_key):

            key = future_to_key[future]
            exception = future.exception()

            if not exception:
                yield key, future.result()
            else:
                yield key, exception


for key, result in fetch_all(s3_object_keys):
    print('key: {}  result: {}'.format(key, result))

Sql credentials stuff

In [ ]:
import json

sql_credentials = json.load(open('/root/thomas/sqlcredentials.json'))

Download data to use for conv pose machine training

In [ ]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import MetaData
from sqlalchemy import Table, select, func, and_, insert, delete, update, or_
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

In [ ]:
# from datetime import datetime
import datetime

login

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                                         sql_credentials["host"], sql_credentials["port"],
                                         sql_credentials["database"]))


In [ ]:
session_class = sessionmaker(bind=engine)
session = session_class()

Base = automap_base()
Base.prepare(engine, reflect=True)

Query all keypoints annotations done on BATI

In [ ]:
keypoints = Base.classes.keypoint_annotations
detections = Base.classes.fish_detections

In [ ]:
results = session.query(detections) \
                 .filter(detections.created_at >= '2019-04-27') \
                 .filter(detections.pen_id == 1) \
                 .filter(detections.site_id == 23) \
                 .all()

In [ ]:
print('Total number of fish detections: {}'.format(len(results)))

In [ ]:
data = []
for d in results:
    tmp = {'left_image_url': d.left_image_url,
           'right_image_url': d.right_image_url,
           'site_id': d.site_id,
           'pen_id': d.pen_id,
            }
    data.append(tmp)

In [ ]:
print(len(results))

Download images

In [ ]:
# !pip3 freeze | grep req

In [ ]:
import datetime
import os
import time

import requests
from tqdm import tqdm

In [ ]:
urls = []

In [ ]:
len(data)

In [ ]:
count = 0
urls = []
for d in tqdm(data):
    # DOWNLOAD LEFT
    url = d['left_image_url']  
    urls.append(url)
    s = url.split('at=')[1].split('/')[0]
    ts = time.mktime(datetime.datetime.strptime(s, "%Y-%m-%dT%H:%M:%S.%f000Z").timetuple())
    date = str(datetime.datetime.utcfromtimestamp(ts).date())
    folder = '/root/data/fish_detections/images/{}/{}/{}/'.format(d['site_id'], d['pen_id'], date)
    if not os.path.isdir(folder):
        os.makedirs(folder)
    image_name = os.path.basename(url).replace('.jpg', str(int(ts)) + '.jpg')
    destination = os.path.join(folder, image_name)
    d['local_path'] = destination
    d['date'] = date
#     if os.path.isfile(destination):
#         continue
#     print(destination)
#     count += 1
#     f = open(destination,'wb')
#     f.write(requests.get(url).content)
#     f.close()
    
    # DOWNLOAD RIGHT
    url = d['right_image_url']   
    urls.append(url)
    s = url.split('at=')[1].split('/')[0]
    ts = time.mktime(datetime.datetime.strptime(s, "%Y-%m-%dT%H:%M:%S.%f000Z").timetuple())
    date = str(datetime.datetime.utcfromtimestamp(ts).date())
    folder = '/root/data/fish_detections/images/{}/{}/{}/'.format(d['site_id'], d['pen_id'], date)
    if not os.path.isdir(folder):
        os.makedirs(folder)
    image_name = os.path.basename(url).replace('.jpg', str(int(ts)) + '.jpg')
    destination = os.path.join(folder, image_name)
    d['local_path'] = destination
    d['date'] = date
#     if os.path.isfile(destination):
#         continue
#     print(destination)
#     count += 1
#     f = open(destination,'wb')
#     f.write(requests.get(url).content)
#     f.close()

In [ ]:
renamed_urls = []
for url in urls:
    url = url.replace('aquabyte-crops', 'aquabyte-frames-resized-inbound')
    renamed_urls.append(url)

In [ ]:
print(len(renamed_urls))

In [ ]:
    with open('/root/data/fish_detections/urls_{}.txt'.format(i), 'w') as f:
        for url in subset:
            f.write("%s\n" % url)

In [ ]:
import numpy as np

In [ ]:
intervals = np.arange(0, len(renamed_urls)+50000, 50000)
intervals

In [ ]:
i = 0
for j in range(1, len(intervals)):
    print(intervals[j])
    if j == len(intervals) -1:
        subset = renamed_urls[intervals[j-1]: ]
#         print('lol')
    else:
        subset = renamed_urls[intervals[j-1]: intervals[j]]
#     print(intervals[j-1], intervals[j])
    with open('/root/data/fish_detections/urls_{}.txt'.format(i), 'w') as f:
        for url in subset:
            f.write("%s\n" % url)
    i += 1

In [ ]:
print(count)

In [ ]:
image_name

Create labelbox format

In [ ]:
for d in data:
    path = d['local_path']
    side = os.path.basename(path).split('_')[0]
    if side == 'left':
        lab_kp = d['left_keypoints']
    else:
        lab_kp = d['right_keypoints']
    
    labels = {}
    for kpp in lab_kp:
        name = kpp['keypointType']
        labels[name] = [{'geometry': {'x': kpp['xCrop'], 'y': kpp['yCrop']}}]
    d['Label'] = labels

In [ ]:
len(data)

In [ ]:
dates = list(set([d['date'] for d in data]))

In [ ]:
dates = ['2019-05-06',
 '2019-04-30',
 '2019-05-03',
 '2019-05-02',
 '2019-05-07',
 '2019-05-05',
 '2019-05-04',
 '2019-04-29',
 '2019-05-01',
 '2019-05-08']

In [ ]:
for dat in dates:
    data_points = [d for d in data if d['date'] == dat]
    print('{}: {}'.format(dat, len(data_points)))
    with open('/root/data/bati/labels/labels_{}.json'.format(dat), 'w') as f:
        json.dump(data_points, f)

In [ ]:
data_points[0]

Check input

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = json.load(open('/root/data/bati/labels/labels.json'))

In [ ]:
from utils.utils import load_image_keypoints, DataGenerator 

In [ ]:
class FLAGS(object):
    """ """
    """
    General settings
    """
    input_size = (512, 512)
    keypoints_order = ["TAIL_NOTCH",
                        "ADIPOSE_FIN",
                        "UPPER_LIP",
                        "ANAL_FIN",
                        "PELVIC_FIN",
                        "EYE",
                        "PECTORAL_FIN",
                        "DORSAL_FIN"]
    augmentation = None
    crop = False
    batch_size = 8
    input_size = (512, 512)
    heatmap_size = 64
    cpm_stages = 3
    joint_gaussian_variance = 1.0
    center_radius = 21
    num_of_joints = 8
    color_channel = 'RGB'
    normalize = True
    use_gpu = True
    gpu_id = 0

In [ ]:
image = cv2.imread(data[0]['local_path'])

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
im, kps = load_image_keypoints(data[0], FLAGS)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(im)
plt.scatter(kps[:, 0], kps[:, 1])
for i in range(8):
    plt.text(kps[i, 0], kps[i,1], FLAGS.keypoints_order[i], color='w')
plt.show()

In [ ]:
train_generator = DataGenerator(data, FLAGS)

In [ ]:
xb, yb, heatmaps = train_generator[10]

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(xb[1, ...])
i = 0
for kpp in yb[1]:
    plt.scatter(kpp[0], kpp[1], c="r")
    plt.text(kpp[0], kpp[1], FLAGS.keypoints_order[i], {"color": "w"})
    i += 1
plt.show()

In [ ]:
for d in data:
    n = (len(list(d['Label'].keys())))
    if n != 8:
        print(n)